# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

In [1]:
from __future__ import print_function, division

import numpy
import scipy.stats
import pandas as pd

import matplotlib.pyplot as pyplot

from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets

# some nicer colors from http://colorbrewer2.org/
COLOR1 = '#7fc97f'
COLOR2 = '#beaed4'
COLOR3 = '#fdc086'
COLOR4 = '#ffff99'
COLOR5 = '#386cb0'

%matplotlib inline

In [51]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')
pd.set_option('display.max_columns', 100)
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,occupbroad,workinschool,email,computerskills,specialskills,firstname,sex,race,h,l,call,city,kind,adid,fracblack,fracwhite,lmedhhinc,fracdropout,fraccolp,linc,col,expminreq,schoolreq,eoe,parent_sales,parent_emp,branch_sales,branch_emp,fed,fracblack_empzip,fracwhite_empzip,lmedhhinc_empzip,fracdropout_empzip,fraccolp_empzip,linc_empzip,manager,supervisor,secretary,offsupport,salesrep,retailsales,req,expreq,comreq,educreq,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,1,0,0,1,0,Allison,f,w,0.0,1.0,0.0,c,a,384.0,0.989360,0.005500,9.527484,0.274151,0.037662,8.706325,1.0,5,,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,6,1,1,1,0,Kristen,f,w,1.0,0.0,0.0,c,a,384.0,0.080736,0.888374,10.408828,0.233687,0.087285,9.532859,0.0,5,,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,1,1,0,1,0,Lakisha,f,b,0.0,1.0,0.0,c,a,384.0,0.104301,0.837370,10.466754,0.101335,0.591695,10.540329,1.0,5,,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,5,0,1,1,1,Latonya,f,b,1.0,0.0,0.0,c,a,384.0,0.336165,0.637370,10.431908,0.108848,0.406576,10.412141,0.0,5,,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,5,1,1,1,0,Carrie,f,w,1.0,0.0,0.0,c,a,385.0,0.397595,0.180196,9.876219,0.312873,0.030847,8.728264,0.0,some,,1.0,9.4,143.0,9.4,143.0,0.0,0.204764,0.727046,10.619399,0.070493,0.369903,10.007352,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [15]:
df = data[['race','call']]
df_b = df[df['race']=='b']
df_w = df[df['race']=='w']
df_b.size, df_w.size

(4870, 4870)

<div class="span5 alert alert-info">
What test is appropriate for this problem? Does CLT apply?
</div>

__Randomization Condition:__ The data must be sampled randomly.

__Independence Assumption:__ The sample values must be independent of each other. This means that the occurrence of one event has no influence on the next event. Usually, if we know that people or items were selected randomly we can assume that the independence assumption is met.

__Sample Size Assumption:__ In general a sample size of 30 is considered sufficient.

CLT applies in this case as the above conditions are met.<br>
We can use 2-sample z-test

<div class="span5 alert alert-info">
What are the null and alternate hypotheses?
</div>

H$_{0}$:  There is no significant difference between proportions<br>
H$_{1}$:  There is significant difference between proportions

<div class="span5 alert alert-info">
Compute margin of error, confidence interval, and p-value.
</div>

In [39]:
#proportion of b receiving calls
prop_b = df_b[df_b['call']==1.0].size/df_b.size
#proportion of w receiving calls
prop_w = df_w[df_w['call']==1.0].size/df_w.size
print('Proportion of b receiving calls: ',prop_b)
print('Proportion of w receiving calls: ',prop_w)

Proportion of b receiving calls:  0.064476386037
Proportion of w receiving calls:  0.0965092402464


In [40]:
#difference in proportions
diff_prop = prop_w-prop_b

#standard deviation
diff_prop_std = numpy.sqrt((prop_b*(1-prop_b)/4870)+(prop_w*(1-prop_w)/4870))
print('Margin of error: ',diff_prop_std)

#95% confidence interval is 'd' distance away from mean
d = 1.96 * diff_prop_std

#confidence interval
ci = (diff_prop-d,diff_prop+d)
print('Confidence Interval: ', ci)

Margin of error:  0.00550367412233
Confidence Interval:  (0.021245652929684569, 0.042820055489206602)


In [43]:
#z-score
#Assuming both proportions to be same according to our hypothesis
#proportion of both b and w receiving calls
prop = df[df['call']==1.0].size/df.size
std = numpy.sqrt((2*prop*(1-prop)/4870))
z_stat = (diff_prop-0)/std    #'0' because null hypothesis states no significant difference between groups
print('z score: ',z_stat)

z score:  5.81017218579


From the z-statistic above, we can reject our null hypothesis<br>
Because z stat $\not\in$ [-1.96, 1.96], which is 95% confidence interval for z.<br>
There is significant difference between proportions

Probability(pvalue) of getting a value z standard deviations away is very low

<div class="span5 alert alert-info">
Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?
</div>

From the analysis, it seems that race is a factor in determining callback success.<br>
But it may not be the only factor. It could be education, experience, age, sex.<br>
Performing analysis based on age groups, experience range and sex might help provide more insight on deciding the important factor.<br>
Comparing z scores of every analysis would be a possible way.